In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout,
                             QPushButton, QLabel, QComboBox, QFileDialog)
from PyQt5.QtWidgets import QHBoxLayout, QSplitter
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
import mplcursors
from matplotlib.gridspec import GridSpec

# 상수 및 설정 값 관리
class Config:
    WAFER_RADIUS = 150000
    SCALE_FACTOR = 1e-7
    SCALE_BAR_LENGTH = 30000
    SCALE_BAR_POSITION = 'lower center'
    SCALE_BAR_LENGTH_PIXELS = 30000
    SCALE_BAR_SIZE_VERTICAL = 500
    TEXT_POSITION_Y = -170000  # 텍스트 추가 위치

# 가로선과 세로선을 계산하는 함수 통합
def calculate_lines(center, pitch, max_die, min_die):
    lines = []
    current = center
    for _ in range(max_die + 2):
        lines.append(current)
        current += pitch
    current = center
    for _ in range(abs(min_die) + 1):
        current -= pitch
        lines.append(current)
    return lines

# 통계 계산 함수 (ABS(Mean) + 3sigma)
def calculate_statistics(values):
    mean_val = np.mean(values)
    sigma_val = np.std(values)
    m3s_val = np.abs(mean_val) + 3 * sigma_val
    m3s_nm = m3s_val * 1e3  # nm 단위로 변환
    return m3s_nm

# 추가적인 통계 지표 표시 함수
def add_statistics_text(ax, data, position=(0, 0)):
    max_val = np.max(data) * 1e3
    min_val = np.min(data) * 1e3
    median_val = np.median(data) * 1e3
    text = f"Max: {max_val:.2f} nm\nMin: {min_val:.2f} nm\nMedian: {median_val:.2f} nm"
    ax.text(position[0], position[1], text, fontsize=10, color='green', ha='center')

# 툴팁 추가 함수
def add_tooltip(quiver):
    cursor = mplcursors.cursor(quiver, hover=True)
    @cursor.connect("add")
    def on_add(sel):
        index = sel.target.index
        x, y = quiver.X[index], quiver.Y[index]
        dx, dy = quiver.U[index], quiver.V[index]
        magnitude = np.sqrt(dx**2 + dy**2) * 1e3  # nm 단위
        sel.annotation.set(text=f"X: {x:.2f}\nY: {y:.2f}\ndX: {dx:.2f}\ndY: {dy:.2f}\nMag: {magnitude:.2f} nm")

# Overlay를 플롯하는 함수
def plot_overlay(ax, x, y, dx, dy, v_lines, h_lines, wafer_radius=Config.WAFER_RADIUS,
                 title='Wafer Vector Map', scale_factor=Config.SCALE_FACTOR,
                 show_statistics=True, unique_id=None):
    # 벡터의 크기를 계산
    magnitudes = np.sqrt(np.array(dx)**2 + np.array(dy)**2)
    magnitudes_nm = magnitudes * 1e3  # nm 단위로 변환

    # 색상 맵 설정
    norm = plt.Normalize(vmin=magnitudes_nm.min(), vmax=magnitudes_nm.max())
    cmap = plt.cm.jet  # 원하는 색상 맵 선택

    quiver = ax.quiver(x, y, dx, dy, angles='xy', scale_units='xy', scale=scale_factor,
                       color=cmap(norm(magnitudes_nm)), width=0.0015, headwidth=3, headlength=3)

    # 컬러바 추가
    cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)
    cbar.set_label('Overlay Error Magnitude (nm)')

    # 기존의 가로선, 세로선, 웨이퍼 경계 등 추가
    ax.axvline(0, color='red', linewidth=1.0)
    ax.axhline(0, color='red', linewidth=1.0)
    for vline in v_lines:
        ax.axvline(vline, color='black', linestyle='--', linewidth=0.8)
    for hline in h_lines:
        ax.axhline(hline, color='black', linestyle='--', linewidth=0.8)
    wafer_circle = Circle((0, 0), wafer_radius, color='green', fill=False,
                          linestyle='-', linewidth=2)
    ax.add_patch(wafer_circle)

    # 통계치 계산 및 텍스트 추가
    if show_statistics:
        mean_plus_3sigma = calculate_statistics(magnitudes)
        ax.text(0, Config.TEXT_POSITION_Y,
                f'|m|+3s: {mean_plus_3sigma:.2f} nm',
                fontsize=10, color='red', ha='center')
        # 추가적인 통계 지표 표시
        add_statistics_text(ax, magnitudes, position=(0, Config.TEXT_POSITION_Y - 30000))

    # UNIQUE_ID를 플롯 제목에 포함
    if unique_id:
        full_title = f"{title} - {unique_id}"
    else:
        full_title = title
    ax.set_title(full_title)

    ax.set_xlabel('Wafer X Coordinate (wf_x)')
    ax.set_ylabel('Wafer Y Coordinate (wf_y)')
    ax.axis('equal')
    ax.grid(False)

    # 툴팁 추가
    add_tooltip(quiver)

    return quiver

# 히트맵을 플롯하는 함수
def plot_heatmap(ax, x, y, z, title):
    # 그리드 생성
    xi = np.linspace(min(x), max(x), 100)
    yi = np.linspace(min(y), max(y), 100)
    xi, yi = np.meshgrid(xi, yi)

    # 데이터 보간
    from scipy.interpolate import griddata
    zi = griddata((x, y), z, (xi, yi), method='cubic')

    # 히트맵 또는 컨투어 플롯
    heatmap = ax.contourf(xi, yi, zi, levels=100, cmap='jet')
    cbar = plt.colorbar(heatmap, ax=ax)
    cbar.set_label('Overlay Error Magnitude (nm)')
    ax.set_title(title)
    ax.set_xlabel('Wafer X Coordinate (wf_x)')
    ax.set_ylabel('Wafer Y Coordinate (wf_y)')
    ax.axis('equal')
    ax.grid(False)

# 스트림라인 플롯 함수
def plot_streamlines(ax, x, y, dx, dy, title):
    xi = np.linspace(min(x), max(x), 100)
    yi = np.linspace(min(y), max(y), 100)
    xi, yi = np.meshgrid(xi, yi)

    from scipy.interpolate import griddata
    ui = griddata((x, y), dx, (xi, yi), method='cubic')
    vi = griddata((x, y), dy, (xi, yi), method='cubic')

    strm = ax.streamplot(xi, yi, ui, vi, density=2, color='blue')
    ax.set_title(title)
    ax.set_xlabel('Wafer X Coordinate (wf_x)')
    ax.set_ylabel('Wafer Y Coordinate (wf_y)')
    ax.axis('equal')
    ax.grid(False)

# 히스토그램 플롯 함수
def plot_histogram(ax, data, title, xlabel):
    data_nm = np.array(data) * 1e3  # nm 단위로 변환
    ax.hist(data_nm, bins=30, color='blue', edgecolor='black')
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Frequency')

# 박스플롯 함수
def plot_boxplot(ax, data, title):
    data_nm = np.array(data) * 1e3  # nm 단위로 변환
    ax.boxplot(data_nm, vert=True)
    ax.set_title(title)
    ax.set_ylabel('Overlay Error Magnitude (nm)')

# 클릭한 플롯을 확대해서 보여주는 클래스
class EnlargedPlotWindow(QMainWindow):
    def __init__(self, x, y, u, v, title, v_lines, h_lines,
                 wafer_radius=Config.WAFER_RADIUS, scale_factor=Config.SCALE_FACTOR,
                 unique_id=None):
        super().__init__()
        # UNIQUE_ID를 창 제목에 포함
        if unique_id:
            self.setWindowTitle(f"{title} - {unique_id}")
        else:
            self.setWindowTitle(title)
        self.setGeometry(100, 100, 1200, 800)

        # 확대된 플롯을 생성
        fig = plt.figure(figsize=(12, 8))
        gs = GridSpec(2, 2, figure=fig)

        # 메인 벡터 플롯
        ax_main = fig.add_subplot(gs[0, 0])
        plot_overlay(ax_main, x, y, u, v, v_lines, h_lines, wafer_radius, title, scale_factor,
                     unique_id=unique_id)

        # 히트맵 플롯
        ax_heatmap = fig.add_subplot(gs[0, 1])
        magnitudes = np.sqrt(np.array(u)**2 + np.array(v)**2)
        magnitudes_nm = magnitudes * 1e3  # nm 단위로 변환
        plot_heatmap(ax_heatmap, x, y, magnitudes_nm, title='Heatmap')

        # 스트림라인 플롯
        ax_stream = fig.add_subplot(gs[1, 0])
        plot_streamlines(ax_stream, x, y, u, v, title='Streamlines')

        # 히스토그램 플롯
        ax_hist = fig.add_subplot(gs[1, 1])
        plot_histogram(ax_hist, magnitudes, title='Overlay Error Distribution', xlabel='Overlay Error Magnitude (nm)')

        # Matplotlib FigureCanvas 생성
        self.canvas = FigureCanvas(fig)

        # Navigation Toolbar 추가
        self.toolbar = NavigationToolbar(self.canvas, self)

        layout = QVBoxLayout()
        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

# 플롯을 보여주는 클래스
class PlotWindow(QMainWindow):
    def __init__(self, unique_id, df):
        super().__init__()
        self.setWindowTitle(f"Plot for {unique_id}")
        self.setGeometry(100, 100, 1200, 800)

        self.unique_id = unique_id  # UNIQUE_ID 저장

        # 데이터 필터링
        df_lot = df[df['UNIQUE_ID'] == unique_id]

        # 데이터 추출
        data = df_lot.to_dict(orient='list')
        wf_x = data['wf_x']
        wf_y = data['wf_y']

        # 스텝 피치 및 맵 시프트 추출
        step_pitch_x = df_lot['STEP_PITCH_X'].iloc[0]
        step_pitch_y = df_lot['STEP_PITCH_Y'].iloc[0]
        map_shift_x = df_lot['MAP_SHIFT_X'].iloc[0]
        map_shift_y = df_lot['MAP_SHIFT_Y'].iloc[0]
        start_left = -(step_pitch_x)/2 + map_shift_x
        start_bottom = -(step_pitch_y)/2 + map_shift_y
        max_die_x = int(df_lot['DieX'].max())
        min_die_x = int(df_lot['DieX'].min())
        max_die_y = int(df_lot['DieY'].max())
        min_die_y = int(df_lot['DieY'].min())

        self.vertical_lines = calculate_lines(start_left, step_pitch_x, max_die_x, min_die_x)
        self.horizontal_lines = calculate_lines(start_bottom, step_pitch_y, max_die_y, min_die_y)

        # 확대된 창들을 저장할 리스트
        self.enlarged_plot_windows = []

        # Figure 생성
        fig = plt.figure(figsize=(12, 8))
        gs = GridSpec(4, 3, figure=fig)
        fig.suptitle(f'Visualizations for Lot {unique_id}', fontsize=16)

        self.quivers = []

        # 플롯 설정 정보를 리스트로 정의
        plot_configs = [
            {'position': gs[0, 0], 'dx': data['X_reg'], 'dy': data['Y_reg'], 'title': 'Raw(X_reg,Y_reg)'},
            {'position': gs[0, 1], 'dx': data['pred_x'], 'dy': data['pred_y'], 'title': 'OSR_Fitting(WK,RK)'},
            {'position': gs[0, 2], 'dx': data['residual_x'], 'dy': data['residual_y'], 'title': 'Residual'},
            {'position': gs[1, 0], 'dx': data['psm_fit_x'], 'dy': data['psm_fit_y'], 'title': 'PSM Input'},
            {'position': gs[1, 1], 'dx': data['residual_x_depsm'], 'dy': data['residual_y_depsm'], 'title': 'Residual(Remove_PSM)'},
            {'position': gs[1, 2], 'dx': data['cpe19p_pred_x'], 'dy': data['cpe19p_pred_y'], 'title': 'CPE 19para Fitting'},
            {'position': gs[2, 0], 'dx': data['cpe19p_resi_x'], 'dy': data['cpe19p_resi_y'], 'title': 'CPE 19para Residual'},
            {'position': gs[2, 1], 'dx': data['ideal_psm_x'], 'dy': data['ideal_psm_y'], 'title': 'Ideal PSM'},
            {'position': gs[2, 2], 'dx': data['delta_psm_x'], 'dy': data['delta_psm_y'], 'title': 'Delta PSM'},
            {'position': gs[3, 0], 'dx': data['delta_psm_x'], 'dy': [0]*len(data['delta_psm_y']), 'title': 'Delta PSM X'},
            {'position': gs[3, 1], 'dx': [0]*len(data['delta_psm_x']), 'dy': data['delta_psm_y'], 'title': 'Delta PSM Y'},
            # 필요에 따라 더 추가 가능
        ]

        for config in plot_configs:
            ax = fig.add_subplot(config['position'])
            quiver = plot_overlay(ax, wf_x, wf_y, config['dx'], config['dy'],
                                  self.vertical_lines, self.horizontal_lines,
                                  title=config['title'], unique_id=self.unique_id)
            self.quivers.append((quiver, config['title']))

        # 사용되지 않는 서브플롯 숨기기 (예: 마지막 빈 플롯)
        ax_empty = fig.add_subplot(gs[3, 2])
        ax_empty.axis('off')

        # Matplotlib FigureCanvas 생성
        self.canvas = FigureCanvas(fig)

        # Navigation Toolbar 추가
        self.toolbar = NavigationToolbar(self.canvas, self)

        layout = QVBoxLayout()
        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # 클릭 이벤트 연결
        self.canvas.mpl_connect('button_press_event', self.on_click)

    def on_click(self, event):
        for quiver, title in self.quivers:
            if event.inaxes and quiver.contains(event)[0]:
                # 클릭한 플롯을 확대하여 새 창으로 표시
                enlarged_window = EnlargedPlotWindow(
                    quiver.X, quiver.Y, quiver.U, quiver.V,
                    title=title,
                    v_lines=self.vertical_lines,
                    h_lines=self.horizontal_lines,
                    unique_id=self.unique_id  # UNIQUE_ID 전달
                )
                self.enlarged_plot_windows.append(enlarged_window)
                enlarged_window.show()

# 메인 윈도우
class MainWindow(QMainWindow):
    def __init__(self, df):
        super().__init__()
        self.setWindowTitle("Unique ID Plot Viewer")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        # 라벨 추가
        label = QLabel("Select a unique_id to view the plot:")
        layout.addWidget(label)

        # 콤보박스 추가
        self.combo_box = QComboBox()
        self.combo_box.addItems(df['UNIQUE_ID'].unique())
        layout.addWidget(self.combo_box)

        # 버튼 추가
        button = QPushButton("Show Plot")
        button.clicked.connect(self.show_plot)
        layout.addWidget(button)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # 데이터 저장
        self.df = df

    def show_plot(self):
        unique_id = self.combo_box.currentText()
        self.plot_window = PlotWindow(unique_id, self.df)
        self.plot_window.show()

# PyQt 애플리케이션 실행
def main():
    app = QApplication([])
    file_dialog = QFileDialog()
    file_path, _ = file_dialog.getOpenFileName(None, "Select CSV File", "", "CSV Files (*.csv)")
    if not file_path:
        return
    # CSV 데이터 로드
    df = pd.read_csv(file_path)

    main_window = MainWindow(df)
    main_window.show()
    app.exec_()

if __name__ == "__main__":
    main()


c:\Python38\lib\site-packages\mplcursors\_pick_info.py:142: UserWarning: Selection.target.index is deprecated and will be removed in the future; use Selection.index instead.
  warnings.warn(
c:\Python38\lib\site-packages\mplcursors\_pick_info.py:142: UserWarning: Selection.target.index is deprecated and will be removed in the future; use Selection.index instead.
  warnings.warn(


RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

<Figure size 1902x951 with 6 Axes>

RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

<Figure size 1182x742 with 23 Axes>

RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

<Figure size 1902x951 with 6 Axes>

RuntimeError: wrapped C/C++ object of type FigureCanvasQTAgg has been deleted

<Figure size 1182x742 with 6 Axes>